
# 小地域の収入推定

```Data```フォルダには、小地域の収入データ(```income_district.csv```)と国勢調査の小地域集計データが格納されています。

国勢調査の小地域集計データ（人口構成、労働力構成、住宅形態など）から、その地域の収入を推定することが考えられます。推定のための機械学習・深層学習モデルを構築しなさい。


- データを観察・理解する上で、データの構造を説明しながら、適切なデータ整形を行いなさい
- データ構造や分析結果に対して、少なくとも二つの図で可視化を行いなさい
- モデルの精度を評価し、できるだけ精度が高いモデルを得るよう、適切な特徴量エンジニアリングやモデル選定の考えもまとめなさい


In [6]:
import pandas as pd

# ファイルの選択
directory = 'Data' # ディレクトリのパス

age_data_path = f'{directory}/h27_age_df.csv'
family_data_path = f'{directory}/h27_family_df.csv'
gender_data_path = f'{directory}/h27_gender_df2.csv'
housing_data_path = f'{directory}/h27_house_df.csv'
house_info_data_path = f'{directory}/h27_house_info_df.csv'
industry_data_path = f'{directory}/h27_indusry_df.csv'
occupation_data_path = f'{directory}/h27_job_df.csv'
labor_data_path = f'{directory}/h27_labor_df.csv'
marriage_data_path = f'{directory}/h27_marriage_df.csv'
employment_status_data_path = f'{directory}/h27_work_status_df.csv'
income_data_path = f'{directory}/income_district.csv'

age_data = pd.read_csv(age_data_path) #年齢層別人口、性別別人口などに関するデータ
family_data = pd.read_csv(family_data_path) #家族に関するデータ 
gender_data = pd.read_csv(gender_data_path) #総人口、男女別人口などに関するデータ
housing_data = pd.read_csv(housing_data_path) #住宅関連データ
house_info_data = pd.read_csv(house_info_data_path) #住居の階数別データ
industry_data = pd.read_csv(industry_data_path) #職業別男女人口データ
occupation_data = pd.read_csv(occupation_data_path) #職業別男女人口データ（職業分類が異なる）
labor_data = pd.read_csv(labor_data_path) #労働状況データ
marriage_data = pd.read_csv(marriage_data_path) #結婚状況データ
employment_status_data = pd.read_csv(employment_status_data_path) #就業形態データ
income_data = pd.read_csv(income_data_path) #地域コードと平均収入

In [7]:
# データフレームを結合する
# age_data,family_data
merged_data = pd.merge(age_data, family_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')

# 全てのデータ
merged_data = pd.merge(merged_data, gender_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, housing_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, house_info_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, industry_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, occupation_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, labor_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, marriage_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, employment_status_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, income_data, left_on=['district_id'], right_on=['area_code'], how='left')

# 結合結果の確認
#merged_data.head()

# データをCSVファイルとして保存
#merged_data.to_csv('your_data_file.csv', index=False)